In [ ]:
import json, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

v1model_dir = "marbret_intent_v1"

tokenizer = AutoTokenizer.from_pretrained(v1model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

with open(f"{model_dir}/label_mapping.json", "r", encoding="utf-8") as f:
    mapping = json.load(f)
label_to_id = mapping["label_to_id"]
id_to_label = {int(k): v for k, v in mapping["id_to_label"].items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_one(text):
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    encoding = {k: v.to(device) for k, v in encoding.items()}
    with torch.no_grad():
        logits = model(**encoding).logits
        pred_id = torch.argmax(logits, dim=-1).item()
    return id_to_label[pred_id]

print(predict_one("حاب نبرا من لادروك عاونوني"))


In [2]:
import joblib
baseline_pipeline = joblib.load("baseline_intent_svm/baseline_pipeline.joblib")
print(baseline_pipeline.predict(["حاب نبرا من لادروك عاونوني"])[0])


Looking for support


c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Lin

In [ ]:
#testing both models, marbret v1 and incontext marbret 
marbretV1 = 